<a href="https://colab.research.google.com/github/amngb95/AI_CyberSecurity/blob/master/ICP8/L5_Encrypted_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [2]:
!pip install syft

     |████████████████████████████████| 225kB 4.8MB/s 
     |████████████████████████████████| 389kB 31.7MB/s 
     |████████████████████████████████| 204kB 41.5MB/s 
     |████████████████████████████████| 256kB 40.0MB/s 
     |████████████████████████████████| 1.4MB 35.2MB/s 
     |████████████████████████████████| 450kB 41.9MB/s 
     |████████████████████████████████| 81kB 27.0MB/s 
     |████████████████████████████████| 276kB 47.2MB/s 
     |████████████████████████████████| 51kB 18.3MB/s 
     |████████████████████████████████| 122kB 41.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

In [3]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0711 23:00:13.769487 140005954062208 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0711 23:00:13.787850 140005954062208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [4]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0711 23:00:18.347896 140005954062208 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:18.357702 140005954062208 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:18.359395 140005954062208 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:18.360944 140005954062208 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:18.362396 140005954062208 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:18.363845 140005954062208 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [8]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.5227739214897156
alice loss 1.2230972051620483
Bob loss 0.2856011390686035
alice loss 0.36513975262641907
Bob loss 0.15630334615707397
alice loss 0.11109364777803421
Bob loss 0.08579438924789429
alice loss 0.035689521580934525
Bob loss 0.04732523113489151
alice loss 0.013144610449671745
Bob loss 0.026319142431020737
alice loss 0.006254538428038359
Bob loss 0.014832488261163235
alice loss 0.004013684578239918
Bob loss 0.008536278270184994
alice loss 0.0031653335317969322
Bob loss 0.005071272607892752
alice loss 0.0027447505854070187
Bob loss 0.0031516326125711203
alice loss 0.0024652420543134212


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [10]:
secure_worker._objects # can you identify those objects in the Secure worker?

{4878783193: Parameter containing:
 tensor([-0.0199], requires_grad=True), 20747257237: Parameter containing:
 tensor([[ 0.6062, -0.0735]], requires_grad=True), 78746364767: Parameter containing:
 tensor([[ 1.0704, -0.0944]], requires_grad=True), 81219144167: Parameter containing:
 tensor([0.0098], requires_grad=True)}

### Step 6: Average The Models

In [11]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([-0.0051])

### Step 7: The complete solution over multiple training sessions

In [12]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0048) Alice:tensor(0.0053)
Bob:tensor(0.0009) Alice:tensor(0.0002)
Bob:tensor(0.0006) Alice:tensor(2.2487e-05)
Bob:tensor(0.0004) Alice:tensor(4.0309e-06)
Bob:tensor(0.0003) Alice:tensor(5.6574e-07)
Bob:tensor(0.0002) Alice:tensor(2.4123e-09)
Bob:tensor(0.0001) Alice:tensor(1.2388e-07)
Bob:tensor(0.0001) Alice:tensor(3.2208e-07)
Bob:tensor(7.8052e-05) Alice:tensor(4.4608e-07)
Bob:tensor(5.8465e-05) Alice:tensor(4.8669e-07)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [14]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0207],
        [0.0165],
        [0.9793],
        [0.9751]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0004)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [16]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [17]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [18]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [19]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [20]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [22]:
shares = encrypt(x=7, n_share=10)
shares

(23520057283192182845339,
 9161920128446273113003,
 5383357846536563230830,
 10181923312396954983084,
 5200276701347406419293,
 10336136704682650952938,
 6158083879703213025795,
 18482624104620655827836,
 23145310052800228167299,
 7133459205075068868205)

In [23]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [25]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(19986947084902076939278, 527995103942120757262, 3225687654916041790188)
(237968027164477062236, 7237655767650500223838, 16265006048945262200656)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [28]:
encode(-0.600)

23740629843760239345664

In [29]:
decode(9323)

9.323

In [30]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (15960331682115544046339, 4334169043352234006663, 3446129118292461439221)
Value of y = (11110961217438911038327, 386594167917184410812, 12243074458404144043084)
Value of sum = (3330663055794215597943, 4720763211269418417475, 15689203576696605482305)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [31]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0711 23:00:19.940360 140005954062208 hook.py:98] Torch was already hooked... skipping hooking process
W0711 23:00:19.943433 140005954062208 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:19.944675 140005954062208 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:19.948581 140005954062208 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:19.950402 140005954062208 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:19.951488 140005954062208 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 23:00:19.952959 140005954062208 base.py:628] Worker bob already exists. Replacing 

<VirtualWorker id:secure_worker #objects:44>

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [33]:
x = x.share(bob, alice, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:43297793544 -> bob:76281857879]
	-> (Wrapper)>[PointerTensor | me:84467143368 -> alice:74843513818]
	-> (Wrapper)>[PointerTensor | me:79470894897 -> secure_worker:21594338039]
	*crypto provider: me*

In [34]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{47262761115: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 47617665562: tensor([[ 0.0072],
         [-0.0027]], grad_fn=<AddmmBackward>), 76281857879: tensor([ 535501007346777265, 1030651093229058427, 1266247491052607154,
         2928090839143525871, 4370731451972689849]), 94650889162: tensor([[0.],
         [0.]], requires_grad=True)}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [35]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:16876914993 -> bob:71428101659]
	-> (Wrapper)>[PointerTensor | me:44766866479 -> alice:46170425838]
	-> (Wrapper)>[PointerTensor | me:15653216179 -> secure_worker:17862295832]
	*crypto provider: me*

In [36]:
bob._objects # bob has another tensor.. why?

{47262761115: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 47617665562: tensor([[ 0.0072],
         [-0.0027]], grad_fn=<AddmmBackward>), 71428101659: tensor([1071002014693554530, 2061302186458116854, 2532494982105214308,
         5856181678287051742, 8741462903945379698]), 76281857879: tensor([ 535501007346777265, 1030651093229058427, 1266247491052607154,
         2928090839143525871, 4370731451972689849]), 94650889162: tensor([[0.],
         [0.]], requires_grad=True)}

In [37]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [39]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [40]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [41]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [42]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [43]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [44]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [46]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:86948000659 -> bob:31626001089]
	-> (Wrapper)>[PointerTensor | me:99086652276 -> alice:73464745041]
	*crypto provider: me*

In [47]:
bob._objects

{31626001089: tensor([3575693038697262748, 3771764617348691817, 2404649825011168334]),
 47262761115: tensor([[0., 0.],
         [0., 1.]], requires_grad=True),
 47617665562: tensor([[ 0.0072],
         [-0.0027]], grad_fn=<AddmmBackward>),
 76281857879: tensor([ 535501007346777265, 1030651093229058427, 1266247491052607154,
         2928090839143525871, 4370731451972689849]),
 94650889162: tensor([[0.],
         [0.]], requires_grad=True)}

In [0]:
y = x + x

In [49]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [73]:
# create cells as needed. Separate your solution into logical cells.
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0711 23:08:01.226560 140005954062208 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
crypto_provider = sy.VirtualWorker(id="crypto_provider", hook=hook)


In [75]:
bob.clear_objects()
ted.clear_objects()
alice.clear_objects()
carol.clear_objects()

<VirtualWorker id:carol #objects:0>

In [76]:
# Clear workers if previously holding any values
print(bob._objects)
print(alice._objects)
print(ted._objects)
print(carol._objects)

{}
{}
{}
{}


In [77]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.],[0,0],[1,1],[0,1],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.],[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data = data.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
target = target.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
print(bob._objects)

{47230014400: tensor([[1528772906833016424, 3875720970750704525],
        [2196823855226619317, 3816609173185229910],
        [2318215699648838570,  618255101211756528],
        [2384204316509117865, 2528086144396084064],
        [2375494772242104988, 2820604938899128698],
        [ 206491903018598440, 4017846778175666179],
        [2140451441715586820, 2415124053219902795],
        [3768596610699898977, 2220643542936225197]]), 36093816022: tensor([[4054236995934485168],
        [1529589170604004631],
        [4545971774870845943],
        [ 484172368652067871],
        [ 983195260020836532],
        [2004013871638418521],
        [1318933031422314844],
        [3171825695828445965]])}


In [0]:
model = nn.Linear(2,1)

In [0]:
model = model.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
opt = optim.SGD(params=model.parameters(), lr=0.1).fix_precision()

In [81]:
print(model.weight)
print(opt)

Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:46901900252 -> bob:77783789125]
	-> (Wrapper)>[PointerTensor | me:40915774547 -> alice:78597865270]
	-> (Wrapper)>[PointerTensor | me:66078166526 -> ted:38160090412]
	-> (Wrapper)>[PointerTensor | me:57822910779 -> carol:46358411961]
	*crypto provider: crypto_provider*
SGD (
Parameter Group 0
    dampening: 0
    lr: FixedPrecisionTensor>tensor([100])
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [85]:

for i in range(10):

    # Train Model'S
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()
    opt.step()
    print(F'loss {loss.get().float_precision()}')

loss tensor([-5.8418e+13])
loss tensor([4.4713e+13])
loss tensor([-1.9850e+13])
loss tensor([1.7804e+13])
loss tensor([-2.5774e+13])
loss tensor([-2.8099e+13])
loss tensor([-7.1662e+11])
loss tensor([4.0725e+12])
loss tensor([3.6354e+13])
loss tensor([-7.6791e+13])


In [86]:
model.weight

Parameter containing:
Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:10641711008 -> bob:25847427763]
	-> (Wrapper)>[PointerTensor | me:68963786698 -> alice:89196484275]
	-> (Wrapper)>[PointerTensor | me:61901602463 -> ted:79390203677]
	-> (Wrapper)>[PointerTensor | me:85773156692 -> carol:22476136711]
	*crypto provider: crypto_provider*

In [87]:
#aggregated model 
model = model.get()
print(model.float_precision().weight)

Parameter containing:
tensor([[0.4990, 0.0630]], requires_grad=True)
